In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
data_dir = 'C:\\Users\Kush\\vscodeProjects\datasets\catvsdog'
#making windows path
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('C:/Users/Kush/vscodeProjects/datasets/catvsdog')

In [4]:
#counting nuumber of images
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

10000


In [5]:
#making dictionary
images_dict = {
    'cat': list(data_dir.glob('cat/*')),
    'dog': list(data_dir.glob('dog/*')),
}

In [6]:
#making labels
labels_dict = {
    'cat' : 0,
    'dog' : 1,
}

In [7]:
#resizing images using cv2
X, y = [], []

for name, images in images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(labels_dict[name])

In [8]:
X = np.array(X)
y = np.array(y)

In [9]:
#splitting data set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [10]:
#scaling images
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [12]:
#data augmentation to improve accuracy
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(180, 180,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1),
  ]
)

In [15]:
#building cnn
num_classes = 2

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'), #2d convulational layer
  layers.MaxPooling2D(), #maxpooling
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2), #dropping random nodes
  layers.Flatten(),
  layers.Dense(128, activation='relu'), #dense layer
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=12) 

Epoch 1/12
235/235 [==============================] - 195s 798ms/step - loss: 0.6941 - accuracy: 0.5527
Epoch 2/12
235/235 [==============================] - 186s 793ms/step - loss: 0.6369 - accuracy: 0.6376
Epoch 3/12
235/235 [==============================] - 186s 790ms/step - loss: 0.5873 - accuracy: 0.6872
Epoch 4/12
235/235 [==============================] - 186s 793ms/step - loss: 0.5534 - accuracy: 0.7156
Epoch 5/12
235/235 [==============================] - 184s 784ms/step - loss: 0.5222 - accuracy: 0.7427
Epoch 6/12
235/235 [==============================] - 184s 783ms/step - loss: 0.4962 - accuracy: 0.7573
Epoch 7/12
235/235 [==============================] - 188s 798ms/step - loss: 0.4719 - accuracy: 0.7761
Epoch 8/12
235/235 [==============================] - 185s 787ms/step - loss: 0.4587 - accuracy: 0.7819
Epoch 9/12
235/235 [==============================] - 185s 787ms/step - loss: 0.4353 - accuracy: 0.7963
Epoch 10/12
235/235 [==============================] - 185s 787m

In [16]:
model.evaluate(X_test_scaled,y_test)

79/79 [==============================] - 14s 142ms/step - loss: 0.4243 - accuracy: 0.8056


[0.4243372678756714, 0.8055999875068665]